# Champion Winrate in Lane

**Task One:** Provide a summarization of champion winrates in all lanes for Masters+ games in Ranked Solo/Duo queue for the current season within the KR and EUW servers.

**Task Two:** Summarize champion lead through match times for every matchup.

We will follow these steps: \
1) Connecting to a summoner profile through Riot Games API and finding the keys needed to find games. \
2) Extracting the champion, match Id, and role of each match played by the player for Ranked Solo/Duo Games. \
3) Repeat step 2 for Masters+ Tier of one server. \
4) Repeat steps 3 for KR and EUW servers.

After steps 2-4, create a CSV file summarizing results to send to University of Washington League of Legends collegiate team.

In [1]:
import requests
import json
import pandas as pd
import time

### Part 0: Background Data 

Within Riot's API for matches, the champions only have Ids and no names associated with them. \
Before we investigate the matches separately, we need to have a dataFrame of all of the champions associated with their id.

In [69]:
resp_chmp = requests.get('http://ddragon.leagueoflegends.com/cdn/10.2.1/data/en_US/champion.json')
r3 = resp_chmp.json()
r3 = r3['data']
chmp_name = []
for i in r3:
    chmp_name.append(i)
x = {}
for i in range(len(r3)):
    name = chmp_name[i]
    x[i]= int(r3[name]['key'])
chmp = pd.DataFrame(chmp_name, columns=['Name'])
chmp = chmp.rename(index=x).sort_index()

### Part 1: Extracting Data per User

Investigating user match winrate data, we will proceed with the following steps: \
1) Find the accountId of the summoner through Riot Games API. \
2) Find the set of matches for a user through Riot Games API. \
3) Make a dataset of champion matchups and who won or lost along with the gameId. \
4) Create a statistical summary of findings. 

In [70]:
# 1) Find the accountId of the summoner through Riot Games API.
# ------------------------------------------------------------------------------
# Returns the accountId of a user using the summonerName and API Key the user has
# Inputs:
#   summonerName: Name within League of Legends, case and character sensitive
#   APIKey: API Key from Riot Games 
def accId(summonerName, APIKey):
    URL = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/" + summonerName + "?api_key=" + APIKey
    response = requests.get(URL)
    r1 = response.json()
    json.dumps(r1, indent = 3, sort_keys=True)
    return r1['accountId']

In [71]:
# 2) Find the set of matches for a user through Riot Games API.
# ------------------------------------------------------------------------------
# Find the accountId to get all of the matches from the summoner.
summonerName = (str)(input('Type your Summoner Name here and DO NOT INCLUDE ANY SPACES: '))
APIKey = (str)(input('Copy and paste your API Key here: '))
summ_accId = accId(summonerName, APIKey)
URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + summ_accId + "?api_key=" + APIKey
response = requests.get(URL)
r1 = response.json()
r1 = r1['matches']
# r1 = (json.dumps(r1, indent = 3, sort_keys=True)) # - only to view

Type your Summoner Name here and DO NOT INCLUDE ANY SPACES: Phume
Copy and paste your API Key here: RGAPI-4945197c-8466-4fa1-8508-4779adbddeaf


In [72]:
# We want to get all of the summonerIds of a queue
names = []
for i in range(0, 3):
    if i == 0:
        league = 'challenger'
    elif i == 1:
        league = 'grandmaster' 
    else:
        league = 'master'
    URL3 = "https://na1.api.riotgames.com/lol/league/v4/" + league + "leagues/by-queue/RANKED_SOLO_5x5?api_key=" + APIKey
    response3 = requests.get(URL3)
    r3 = response3.json()
    r3 = r3['entries']
    for i in range(len(r3)):
        names.append(accId(r3[i]['summonerName'], APIKey))
print(names)

KeyError: 'accountId'

In [41]:
# Returns a summary of all solo queue matches from the summoner with their champion and gameId
summ = pd.DataFrame(columns =  ["region", "gameId", "lane", "role", "Name"])
for i in range(len(r1)):
    if r1[i]['queue'] == 420:
        platformId = r1[i]['platformId']
        gameId = r1[i]['gameId'] 
        lane = r1[i]['lane']
        role = r1[i]['role']
        arr = chmp.loc[chmp['ID'] == str(r1[i]['champion'])]['Name'].array
        name = arr[0]
        hold = pd.Series([platformId, gameId, lane, role, name], index = ["region", "gameId", "lane", "role", "Name"])
        summ = summ.append(hold, ignore_index = True)

# We need to dive into the game and find the champion they are going against as well. We only have the championname.
# Use the gameId. Limited by number of API calls though for every gameId, so make sure unique.

In [118]:
len(summ['gameId'])

71

In [126]:
# Works for the lengh
mtchup = pd.DataFrame(columns = ["Champ1", "Champ2", "Win_Champ1", "Win_Champ2", "role"])
for j in range(len(summ['gameId'])):
    gameId = summ['gameId'][j]
    URL_mch = "https://na1.api.riotgames.com/lol/match/v4/matches/" + str(gameId) + "?api_key=" + APIKey
    response = requests.get(URL_mch)
    r1 = response.json()
    r1 = r1['participants']
    match = pd.DataFrame(columns =  ["champ", "win", "lane", "role"])
    for i in range(len(r1)):
        arr = chmp.loc[chmp['ID'] == str(r1[i]['championId'])]['Name'].array
        name = arr[0]
        win = r1[i]['stats']['win']
        lane = r1[i]['timeline']['lane']
        role = r1[i]['timeline']['role']
        if role == "DUO_SUPPORT":
            role = "SUPPORT"
        elif role == "DUO_CARRY":
            role = "BOTTOM_CARRY"
        elif role == "NONE":
            role = "JUNGLE"
        elif role == "SOLO" and lane == "MIDDLE":
            role = "MID"
        else:
            role = "TOP"
        entry = pd.Series([name, win, lane, role], index = ["champ", "win", "lane", "role"])
        match = match.append(entry, ignore_index = True)
    # Puts it into an order where you can put in the values together by indexing over match
    match = match.sort_values(by=['lane', 'role', 'win'])
    for i in range(0, len(match), 2):
        Champ1 = match.iloc[i]['champ']
        Champ2 = match.iloc[i + 1]['champ'] # we always know the second champ won
        role = match.iloc[i]['role']
        name = Champ1[0:2] + Champ2[0:2] + role[0:2]
        # If there is a value of Champ1, champ2, then you increment the win_champ2 by 1
        if not(mtchup[(mtchup.Champ1 == Champ1) & (mtchup.Champ2 == Champ2) & (mtchup.role == role)].empty):
            mtchup.at[name, 'Win_Champ2'] = mtchup.at[name, 'Win_Champ2'] + 1
        # If there is a value of Champ1, champ2, but swapped, then you incrmeent the win_champ1 by 1
        elif not(mtchup[(mtchup.Champ1 == Champ1) & (mtchup.Champ2 == Champ2) & (mtchup.role == role)].empty):
            mtchup.at[name, 'Win_Champ1'] = mtchup.at[name, 'Win_Champ1'] + 1
        # There is no current entry so you create a new series to append into the matchup and set the values as necessary
        else:
            score = pd.Series([Champ1, Champ2, 0, 1, role], index = ["Champ1", "Champ2", "Win_Champ1", "Win_Champ2", "role"]).rename(name)
            mtchup = mtchup.append(score)

         Champ1    Champ2 Win_Champ1 Win_Champ2          role
DrVeBO   Draven    Veigar          0          1  BOTTOM_CARRY
LeSoSU    Leona    Soraka          0          1       SUPPORT
KaJaJU  Karthus  JarvanIV          0          1        JUNGLE
SwSyMI    Swain    Syndra          0          1           MID
RuYoTO   Rumble    Yorick          0          1           TOP
...         ...       ...        ...        ...           ...
DiSiTO    Diana    Singed          0          1           TOP
DrEzBO   Draven    Ezreal          0          1  BOTTOM_CARRY
ReKaJU   RekSai   Karthus          0          1        JUNGLE
FiVlMI     Fizz  Vladimir          0          1           MID
AkViTO    Akali    Viktor          0          1           TOP

[336 rows x 5 columns]


In [127]:
file_path=r'winrates.xlsx'
sum_stats=pd.ExcelWriter(file_path)
mtchup.to_excel(sum_stats)
sum_stats.save()
sum_stats.close()

In [43]:
# Doing for testing
gameId = "3225011906"
URL_mch = "https://na1.api.riotgames.com/lol/match/v4/matches/" + gameId + "?api_key=" + APIKey
# Looking at only 1 game
response = requests.get(URL_mch)
r1 = response.json()
r1 = r1['participants']
match = pd.DataFrame(columns =  ["champ", "win", "lane", "role"])
for i in range(len(r1)):
    arr = chmp.loc[chmp['ID'] == str(r1[i]['championId'])]['Name'].array
    name = arr[0]
    win = r1[i]['stats']['win']
    lane = r1[i]['timeline']['lane']
    role = r1[i]['timeline']['role']
    if role == "DUO_SUPPORT":
        role = "SUPPORT"
    elif role == "DUO_CARRY":
        role = "BOTTOM_CARRY"
    elif role == "NONE":
        role = "JUNGLE"
    elif role == "SOLO" and lane == "MIDDLE":
        role = "MID"
    else:
        role = "TOP"
    entry = pd.Series([name, win, lane, role], index = ["champ", "win", "lane", "role"])
    match = match.append(entry, ignore_index = True)
# Puts it into an order where you can put in the values together by indexing over match
match = match.sort_values(by=['lane', 'role', 'win'])
print(match)


      champ    win    lane          role
5    Draven  False  BOTTOM  BOTTOM_CARRY
4    Veigar   True  BOTTOM  BOTTOM_CARRY
9     Leona  False  BOTTOM       SUPPORT
3    Soraka   True  BOTTOM       SUPPORT
6   Karthus  False  JUNGLE        JUNGLE
1  JarvanIV   True  JUNGLE        JUNGLE
8     Swain  False  MIDDLE           MID
2    Syndra   True  MIDDLE           MID
7    Rumble  False     TOP           TOP
0    Yorick   True     TOP           TOP


In [111]:
# This is the internal part of a for loop that encompasses looking through 
mtchup = pd.DataFrame(columns = ["Champ1", "Champ2", "Win_Champ1", "Win_Champ2", "role"])
for i in range(0, len(match), 2):
    Champ1 = match.iloc[i]['champ']
    Champ2 = match.iloc[i + 1]['champ'] # we always know the second champ won
    role = match.iloc[i]['role']
    name = Champ1[0:2] + Champ2[0:2] + role[0:2]
    # If there is a value of Champ1, champ2, then you increment the win_champ2 by 1
    if not(mtchup[(mtchup.Champ1 == Champ1) & (mtchup.Champ2 == Champ2)].empty):
        mtchup.at[name, 'Win_Champ2'] = mtchup.at[name, 'Win_Champ2'] + 1
    # If there is a value of Champ1, champ2, but swapped, then you incrmeent the win_champ1 by 1
    elif not(mtchup[(mtchup.Champ1 == Champ1) & (mtchup.Champ2 == Champ2)].empty):
        mtchup.at[name, 'Win_Champ1'] = mtchup.at[name, 'Win_Champ1'] + 1
    # There is no current entry so you create a new series to append into the matchup and set the values as necessary
    else:
        score = pd.Series([Champ1, Champ2, 0, 1, role], index = ["Champ1", "Champ2", "Win_Champ1", "Win_Champ2", "role"]).rename(name)
        mtchup = mtchup.append(score)
print(mtchup)

         Champ1    Champ2 Win_Champ1 Win_Champ2          role
DrVeBO   Draven    Veigar          0          1  BOTTOM_CARRY
LeSoSU    Leona    Soraka          0          1       SUPPORT
KaJaJU  Karthus  JarvanIV          0          1        JUNGLE
SwSyMI    Swain    Syndra          0          1           MID
RuYoTO   Rumble    Yorick          0          1           TOP


In [274]:
val = match[(match.lane == 'BOTTOM') & (match.role == 'SUPPORT')]
Champ1 = val.iloc[0]['champ']
Champ2 = val.iloc[1]['champ']
if val.iloc[0]['win']:
    win = 'Champ1'
else:
    win = 'Champ2'
val.iloc[0]
score = pd.Series([Champ1, Champ2, win], index = ["Champ1", "Champ2", ])

True

In [263]:
# Table with all of the values of pairing put in for one role
mtchup = pd.DataFrame(columns = ["Champ1", "Champ2", "Win_Champ1", "Win_Champ2"])
for i in range(len(chmp)):
    for j in range(len(chmp)):
        one = chmp.iloc[j]['Name']
        two = chmp.iloc[i]['Name']               
        if one != two: # don't want mirror matchups for solo/duo
            if mtchup[((mtchup.Champ1 == one) & (mtchup.Champ2 == two)) | ((mtchup.Champ1 == two) & (mtchup.Champ2 == one))].empty:
                pairing = pd.Series([one, two], index = ["Champ1", "Champ2"])
                mtchup = mtchup.append(pairing, ignore_index = True) 
mtchup = mtchup.fillna(0.0)
print(mtchup)

        Champ1  Champ2  Win_Champ1  Win_Champ2
0         Ahri  Aatrox         0.0         0.0
1        Akali  Aatrox         0.0         0.0
2      Alistar  Aatrox         0.0         0.0
3        Amumu  Aatrox         0.0         0.0
4       Anivia  Aatrox         0.0         0.0
...        ...     ...         ...         ...
10726      Zoe   Ziggs         0.0         0.0
10727     Zyra   Ziggs         0.0         0.0
10728      Zoe  Zilean         0.0         0.0
10729     Zyra  Zilean         0.0         0.0
10730     Zyra     Zoe         0.0         0.0

[10731 rows x 4 columns]


In [ ]:
# Testing for time as there is limitation on API
t_end = time.time() + 60 * 15
while time.time() < t_end:
    # do whatever you do
# This will run for 15 min x 60 s = 900 seconds.
# Function time.time returns the current time in seconds since 1st Jan 1970.
# https://stackoverflow.com/questions/3773869/limit-loop-to-n-iterations-per-minute